<a href="https://colab.research.google.com/github/Guille495/Graph-WaveNet/blob/master/split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget -r --no-parent --no-check-certificate 'https://drive.google.com/uc?export=download&id=1oBoybc4Be4hM2au1AECUFiCRblLVWCzo' -O ./pivot_tx.csv
!wget -r --no-parent --no-check-certificate 'https://drive.google.com/uc?export=download&id=14wcUBZ7MtxeTTRHdz02Xr---aOkBoy4D' -O ./melt_tx_pp.csv

will be placed in the single file you specified.

--2023-01-12 00:21:29--  https://drive.google.com/uc?export=download&id=1oBoybc4Be4hM2au1AECUFiCRblLVWCzo
Resolving drive.google.com (drive.google.com)... 142.250.141.101, 142.250.141.102, 142.250.141.100, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/csfnnfgap4sa53st7jtitdpgm02kf61e/1673482875000/00136855323615452105/*/1oBoybc4Be4hM2au1AECUFiCRblLVWCzo?e=download&uuid=343226de-1f74-4fb7-9d98-c62aa3f2ea3d [following]
--2023-01-12 00:21:31--  https://doc-08-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/csfnnfgap4sa53st7jtitdpgm02kf61e/1673482875000/00136855323615452105/*/1oBoybc4Be4hM2au1AECUFiCRblLVWCzo?e=download&uuid=343226de-1f74-4fb7-9d98-c62aa3f2ea3d
Resolving doc-08-2s-docs.googleusercontent.com (doc-08-2

In [ ]:
!mkdir output

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Load dataset
df = pd.read_csv('./pivot_tx.csv', index_col='date')
df_melt = pd.read_csv('./melt_tx_pp.csv', index_col='date')
df_melt = df_melt.drop(columns=['Unnamed: 0'])


In [ ]:
def generate_graph_seq2seq_io_data(
        df, x_offsets, y_offsets, scaler=None, df_pp = None, add_month=False, add_season=False, add_day=False, pp_list = None
):
    """
    Generate samples from
    :param df:
    :param x_offsets:
    :param y_offsets:
    :param scaler:
    :param add_month:
    :return:
    # x: (epoch_size, input_length, num_nodes, input_dim)
    # y: (epoch_size, output_length, num_nodes, output_dim)
    """
    dates = [pd.to_datetime(i).strftime('%Y-%m-%d') for i in df.index]
    stations = df.columns
    num_samples, num_nodes = df.shape
    data = np.expand_dims(df.values, axis=-1)
    
    data_pp = df_pp.pivot(columns='station_id', values='pp')
    # data_pp = data_pp.reindex(all_dates).astype(float).fillna(method='ffill').fillna(method='bfill')
    data_pp = data_pp.fillna(0)
    data_pp = data_pp.loc[df.index]
    data_pp = np.expand_dims(data_pp.values, axis=-1)
    data_pp = (data_pp - data_pp.min())/(data_pp.max() - data_pp.min())
    data = np.concatenate([data, data_pp], axis=2)

    feature_list = [data]

    # Edit here to add features

    if (add_month):
      months = [(pd.to_datetime(i).month - 1)/ 11 for i in df.index]
      month_of_day = np.tile(months, [1, num_nodes, 1]).transpose((2, 1, 0))
      feature_list.append(month_of_day)
    
    if(add_season):
      season = [(pd.to_datetime(i).month % 4)/ 3 for i in df.index]
      season = np.tile(season, [1, num_nodes, 1]).transpose((2, 1, 0))
      feature_list.append(season)


    if(add_day):
      days_of_year = [(pd.to_datetime(i).day_of_year - 1)/355 for i in df.index]
      days_of_year = np.tile(days_of_year, [1, num_nodes, 1]).transpose((2, 1, 0))
      feature_list.append(days_of_year)

    if(pp_list):
      pp_feature = [(pp_list[i] - min(pp_list))/(max(pp_list)-min(pp_list)) for i in df.index]
      pp_feature = np.tile(pp_feature, [1, num_nodes, 1]).transpose((2, 1, 0))
      feature_list.append(pp_feature)


    data = np.concatenate(feature_list, axis=-1)
    x, y, dates = [], [], []
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive


    for t in range(min_t, max_t):  # t is the index of the last observation.        
        x.append(data[t + x_offsets, ...])
        y.append(data[t + y_offsets, ...])
        dates.append(df.index[t + y_offsets, ...])

    x = np.stack(x, axis=0)
    y = np.stack(y, axis=0)

    return x, y, dates, stations


In [ ]:
def split_data_by_years(df_data, train_years, val_years): # sustituye la funcion generate_train_val_test() de la linea 52 --> https://github.com/Guille495/Graph-WaveNet/blob/master/generate_training_data.py
  total_rows = df_data.shape[0]
  print("total rows",total_rows)
  val_max_year =  sum([train_years, val_years])
  
  total_years = list(set([int(pd.to_datetime(i).strftime('%Y')) for i in df_data.index]))
  
  
  train_years_list = total_years[0:train_years] # List of 26 years
  val_years_list = total_years[train_years:val_max_year] # List of next 8 years
  test_years_list = total_years[val_max_year:] # List of next 8 years


  print("total train", len(train_years_list))
  print("total test", len(test_years_list))
  print("total val", len(val_years_list))
  
  # List of indexes that corresponds to each year
  train_index =  list(filter(lambda i: int(pd.to_datetime(i).strftime('%Y')) in train_years_list, df_data.index))
  test_index =  list(filter(lambda i: int(pd.to_datetime(i).strftime('%Y')) in test_years_list, df_data.index))
  val_index =  list(filter(lambda i: int(pd.to_datetime(i).strftime('%Y')) in val_years_list, df_data.index))


  df_train = df_data[df_data.index.isin(train_index)]
  df_test = df_data[df_data.index.isin(test_index)]
  df_val = df_data[df_data.index.isin(val_index)]

  return df_train, df_test, df_val



In [ ]:
df_train, df_test, df_val = split_data_by_years(df, 26, 8)

total rows 15340
total train 26
total test 8
total val 8


In [ ]:
display(df_melt)

,station_id,dateMMDD,tx,pp
date,,,,
1977-01-01,11000,01-01,31.593750,0.000000
1977-01-01,11001,01-01,23.000000,0.000000
1977-01-01,11002,01-01,22.000000,0.000000
1977-01-01,11005,01-01,24.406250,0.000000
1977-01-01,11006,01-01,30.593750,0.000000
...,...,...,...,...
2018-12-31,87828,12-31,33.406250,0.000000
2018-12-31,87860,12-31,28.906250,0.000000
2018-12-31,87909,12-31,26.406250,0.000000


In [ ]:
df_train_melt, df_test_melt, df_val_melt = split_data_by_years(df_melt, 26, 8)

total rows 2101580
total train 26
total test 8
total val 8


In [ ]:
display(df)

,11000,11001,11002,11005,11006,11007,11008,11011,11012,11013,...,87765,87774,87784,87791,87803,87828,87860,87909,87925,87934
date,,,,,,,,,,,,,,,,,,,,,
1977-01-01,31.59375,23.000000,22.000000,24.40625,30.593750,28.796875,31.000000,24.593750,32.500000,27.000000,...,17.906250,24.000000,33.687500,33.187500,13.203125,27.000000,23.796875,15.601562,13.703125,12.101562
1977-01-02,31.50000,28.703125,22.000000,24.40625,30.406250,27.000000,32.187500,24.593750,34.187500,23.593750,...,14.296875,21.000000,31.406250,29.796875,14.398438,21.203125,18.796875,14.101562,14.703125,11.398438
1977-01-03,25.40625,25.406250,22.000000,24.40625,30.203125,29.000000,28.796875,24.593750,28.406250,26.906250,...,16.000000,20.000000,23.203125,21.296875,14.000000,21.203125,19.296875,15.898438,15.601562,11.203125
1977-01-04,29.00000,27.000000,22.000000,24.40625,30.000000,31.406250,30.406250,24.593750,31.593750,25.593750,...,15.601562,20.000000,27.000000,23.296875,16.203125,23.796875,17.000000,13.796875,13.500000,10.101562
1977-01-05,29.59375,28.796875,22.000000,24.40625,28.796875,31.000000,30.000000,24.593750,30.203125,28.796875,...,19.406250,23.500000,30.500000,26.203125,18.796875,27.406250,16.093750,12.203125,13.898438,10.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,34.40625,31.703125,30.406250,30.00000,32.500000,31.406250,33.406250,27.203125,34.312500,29.000000,...,20.203125,24.796875,25.593750,27.000000,20.406250,28.203125,23.500000,19.906250,21.500000,18.796875
2018-12-28,35.18750,26.703125,28.203125,34.40625,37.593750,31.406250,36.187500,28.593750,36.000000,34.500000,...,20.500000,27.000000,25.593750,30.000000,20.203125,28.000000,27.593750,24.203125,24.500000,15.601562
2018-12-29,35.81250,28.796875,31.203125,33.81250,34.687500,31.406250,30.406250,27.796875,35.093750,34.500000,...,15.296875,20.703125,25.593750,32.812500,12.601562,30.000000,23.406250,22.406250,22.000000,17.796875


In [ ]:
df_train.to_csv('./output/train_pivot.csv')
df_test.to_csv('./output/test_pivot.csv') 
df_val.to_csv('./output/val_pivot.csv') 

In [ ]:
df_train_melt = df_train_melt.reset_index()
df_test_melt = df_test_melt.reset_index()
df_val_melt = df_val_melt.reset_index()

In [ ]:
df_train_pivot_nans = df_train_melt.pivot(index='date', columns='station_id', values='tx')
df_test_pivot_nans = df_test_melt.pivot(index='date', columns='station_id', values='tx')
df_val_pivot_nans = df_val_melt.pivot(index='date', columns='station_id', values='tx')

In [ ]:
df_train_melt.to_csv('./output/train_melt.csv')
df_test_melt.to_csv('./output/test_melt.csv') 
df_val_melt.to_csv('./output/val_melt.csv') 

df_train_pivot_nans.to_csv('./output/train_nans_pivot.csv')
df_test_pivot_nans.to_csv('./output/test_nans_pivot.csv') 
df_val_pivot_nans.to_csv('./output/val_nans_pivot.csv') 

In [ ]:
dfs = { "train": df_train, "test": df_test, "val": df_val  }

In [ ]:
df_pp = df_melt.filter(items=['station_id', 'pp'])

In [ ]:
x_window = 15
y_window = 1

x_offsets = np.sort(np.concatenate((np.arange(-(x_window - 1), 1, 1),)))
y_offsets = np.sort(np.arange(1, (y_window + 1), 1))

# Divide the datasets in x and y and transform the input
for cat in ["train", "val", "test"]:
  x, y, dates, stations = generate_graph_seq2seq_io_data(
        df=dfs[cat],
        x_offsets=x_offsets,
        y_offsets=y_offsets,
        scaler=None,
        df_pp=df_pp,
        add_month=True,
        add_season=True,
        add_day=True
    )

  print(cat, "x: ", x.shape, "y:", y.shape, "date", len(dates))

  np.savez_compressed(
      os.path.join("./output", f"{cat}.npz"),
      x=x,
      y=y,
      x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
      y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
      dates=dates,
      stations=stations
  )

train x:  (9481, 15, 137, 5) y: (9481, 1, 137, 5) date 9481
val x:  (2907, 15, 137, 5) y: (2907, 1, 137, 5) date 2907
test x:  (2907, 15, 137, 5) y: (2907, 1, 137, 5) date 2907


In [ ]:
display(dfs["test"].head(15))

,11000,11001,11002,11005,11006,11007,11008,11011,11012,11013,...,87765,87774,87784,87791,87803,87828,87860,87909,87925,87934
date,,,,,,,,,,,,,,,,,,,,,
2003-01-01,31.000000,22.593750,24.593750,27.796875,23.593750,27.796875,29.000000,23.500000,29.000000,22.906250,...,26.500000,22.00000,23.500000,29.593750,27.000000,25.406250,23.000000,19.296875,21.500000,16.906250
2003-01-02,30.203125,23.406250,25.296875,26.796875,27.500000,26.406250,28.093750,24.296875,30.296875,27.593750,...,25.093750,27.00000,26.906250,29.500000,24.203125,29.796875,25.796875,23.093750,17.796875,14.601562
2003-01-03,25.296875,28.203125,28.296875,27.203125,29.000000,26.906250,27.703125,27.593750,24.593750,27.500000,...,22.406250,26.50000,24.906250,26.593750,21.593750,24.500000,20.093750,22.703125,15.796875,14.601562
2003-01-04,31.796875,26.796875,28.906250,30.593750,31.203125,31.593750,31.000000,27.000000,32.000000,29.796875,...,22.093750,25.00000,34.000000,31.406250,20.000000,29.000000,25.203125,23.093750,16.703125,14.601562
2003-01-05,30.703125,32.000000,30.093750,31.203125,30.000000,33.000000,32.000000,32.000000,33.406250,32.687500,...,22.703125,25.50000,25.093750,24.000000,21.593750,21.593750,20.406250,18.796875,15.601562,12.101562
2003-01-06,31.796875,28.000000,28.203125,28.796875,29.406250,29.203125,30.203125,27.593750,31.000000,28.093750,...,31.296875,31.00000,25.593750,25.296875,27.796875,23.000000,23.296875,21.093750,18.593750,14.703125
2003-01-07,33.812500,21.796875,23.906250,29.406250,30.796875,30.000000,32.000000,22.203125,33.500000,27.203125,...,27.593750,32.00000,31.000000,29.796875,28.796875,34.906250,25.093750,29.203125,18.406250,15.398438
2003-01-08,34.906250,31.000000,29.703125,30.796875,32.500000,31.593750,33.500000,28.703125,33.406250,30.203125,...,22.203125,26.00000,28.406250,29.593750,20.593750,30.203125,27.703125,21.703125,20.500000,16.796875
2003-01-09,35.812500,28.593750,31.500000,33.593750,33.406250,32.593750,34.593750,29.000000,33.406250,33.000000,...,21.093750,26.00000,28.093750,28.296875,20.796875,29.000000,26.703125,24.703125,21.203125,16.703125


,11000,11001,11002,11005,11006,11007,11008,11011,11012,11013,...,87765,87774,87784,87791,87803,87828,87860,87909,87925,87934
date,,,,,,,,,,,,,,,,,,,,,
1977-01-01,31.593750,23.000000,22.000000,24.40625,30.593750,28.796875,31.000000,24.59375,32.500000,27.000000,...,17.906250,24.0,33.687500,33.187500,13.203125,27.000000,23.796875,15.601562,13.703125,12.101562
1977-01-02,31.500000,28.703125,22.000000,24.40625,30.406250,27.000000,32.187500,24.59375,34.187500,23.593750,...,14.296875,21.0,31.406250,29.796875,14.398438,21.203125,18.796875,14.101562,14.703125,11.398438
1977-01-03,25.406250,25.406250,22.000000,24.40625,30.203125,29.000000,28.796875,24.59375,28.406250,26.906250,...,16.000000,20.0,23.203125,21.296875,14.000000,21.203125,19.296875,15.898438,15.601562,11.203125
1977-01-04,29.000000,27.000000,22.000000,24.40625,30.000000,31.406250,30.406250,24.59375,31.593750,25.593750,...,15.601562,20.0,27.000000,23.296875,16.203125,23.796875,17.000000,13.796875,13.500000,10.101562
1977-01-05,29.593750,28.796875,22.000000,24.40625,28.796875,31.000000,30.000000,24.59375,30.203125,28.796875,...,19.406250,23.5,30.500000,26.203125,18.796875,27.406250,16.093750,12.203125,13.898438,10.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002-12-27,34.500000,32.000000,33.812500,34.40625,34.687500,35.187500,35.000000,32.40625,35.000000,35.000000,...,17.093750,21.0,29.906250,28.593750,16.000000,27.000000,24.500000,20.796875,17.000000,14.601562
2002-12-28,28.703125,25.593750,25.796875,32.59375,26.796875,24.000000,22.906250,27.09375,22.796875,26.593750,...,21.593750,23.0,23.093750,23.000000,20.406250,24.000000,20.093750,19.703125,18.000000,14.898438
2002-12-29,29.593750,25.406250,25.593750,32.59375,25.000000,27.593750,25.203125,26.00000,26.203125,28.703125,...,24.093750,18.0,22.500000,26.406250,23.000000,29.000000,25.796875,22.406250,18.593750,13.601562
